# [M_00] PROTOKÓŁ: INICJACJA INFRASTRUKTURY (HACKATHON EDITION)

**PROJEKT:** OMNI-OPERATOR-V1  
**ORGANIZACJA:** [KUŹNIA OPERATORÓW](https://takzenai-hub.pl)  
**STATUS:** INICJACJA_SYSTEMU

Ten notatnik stanowi punkt startowy projektu. Jego zadaniem jest przygotowanie środowiska programistycznego oraz infrastruktury kontenerowej w katalogu głównym projektu (ROOT).

**Wymagania wstępne:**
* Zainstalowany `uv` (menedżer pakietów).
* Zainstalowany Docker Desktop.

## 1. Korekta ścieżek operacyjnych
Ponieważ pracujemy wewnątrz folderu `notebooks/`, musimy przesunąć kontekst operacyjny o jeden poziom wyżej, aby pliki konfiguracyjne (`pyproject.toml`, `.env`, `docker-compose.yml`) trafiły do głównego katalogu projektu.

In [1]:
import os
import sys

# Przejście do katalogu ROOT (rodzic folderu notebooks)
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

print(f"LOG: Bieżący katalog roboczy (ROOT): {os.getcwd()}")

# Rejestracja ścieżki src dla przyszłych importów
sys.path.append(os.path.join(os.getcwd(), "src"))

LOG: Bieżący katalog roboczy (ROOT): c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## 2. Inicjalizacja Struktury Projektu (Clean Architecture)
Tworzymy szkielet katalogów dla kodu źródłowego (`src`) oraz logiki systemowej.

In [2]:
# Foldery wymagane przez architekturę systemu
src_folders = [
    "src/agents",
    "src/core",
    "src/services",
    "src/api",
    "docker"
]

for folder in src_folders:
    os.makedirs(folder, exist_ok=True)
    # Dodanie plików __init__.py dla modułów Pythona
    if folder.startswith("src"):
        init_file = os.path.join(folder, "__init__.py")
        if not os.path.exists(init_file):
            with open(init_file, "w") as f: pass

print("LOG: Struktura katalogów src/ została pomyślnie utworzona.")

LOG: Struktura katalogów src/ została pomyślnie utworzona.


## 3. Konfiguracja Stacku i Zależności (`pyproject.toml`)
Definiujemy "Stack Suwerenności" przy użyciu `uv`. Wykorzystujemy PydanticAI 1.39.0 oraz Langfuse v2.

In [3]:
pyproject_content = """
[project]
name = "OMNI-OPERATOR-V1"
version = "0.1.0"
description = "Autonomiczna fabryka dystrybucji treści - Gemini Hackathon"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
    "pydantic-ai==1.39.0",
    "google-generativeai>=0.8.6",
    "fastapi>=0.128.0",
    "uvicorn>=0.40.0",
    "qdrant-client>=1.16.2",
    "langfuse<3.0.0",
    "moviepy>=2.2.1",
    "python-dotenv>=1.2.1",
    "pydantic-settings>=2.4.0",
    "torch",
    "torchvision",
    "torchaudio",
]

[tool.uv]
managed = true
package = false
"""

with open("pyproject.toml", "w", encoding="utf-8") as f:
    f.write(pyproject_content.strip())

print("LOG: Plik pyproject.toml wygenerowany w ROOT.")

LOG: Plik pyproject.toml wygenerowany w ROOT.


## 4. Infrastruktura Docker (Stability Protocol)
Generujemy plik `docker-compose.yml` zawierający bazę Qdrant, Postgres oraz monitoring Langfuse v2.

In [4]:
docker_compose_v2 = """
version: '3.8'
services:
  langfuse:
    image: langfuse/langfuse:2
    container_name: omni_langfuse
    depends_on:
      db: { condition: service_healthy }
    ports:
      - "3000:3000"
    environment:
      - DATABASE_URL=postgresql://postgres:password@db:5432/langfuse
      - NEXTAUTH_URL=http://localhost:3000
      - NEXTAUTH_SECRET=mysecret
      - SALT=mysalt
    restart: always

  db:
    image: postgres:16-alpine
    container_name: omni_db
    environment:
      - POSTGRES_USER=postgres
      - POSTGRES_PASSWORD=password
      - POSTGRES_DB=langfuse
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U postgres"]
      interval: 3s
      timeout: 3s
      retries: 5

  qdrant:
    image: qdrant/qdrant:v1.12.0
    container_name: omni_qdrant
    ports:
      - "6333:6333"
    volumes:
      - ./qdrant_data:/qdrant/storage
"""

with open("docker-compose.yml", "w", encoding="utf-8") as f:
    f.write(docker_compose_v2.strip())

print("LOG: docker-compose.yml wygenerowany w ROOT.")

LOG: docker-compose.yml wygenerowany w ROOT.


## 5. Konfiguracja Systemowa i Synchronizacja `uv`
Tworzymy plik `src/core/config.py` oraz szablon zmiennych środowiskowych.

In [ ]:
# 1. Tworzenie src/core/config.py
config_code = """
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file=".env", extra="ignore")

    gemini_api_key: str = Field(alias="GEMINI_API_KEY")
    langfuse_public_key: str = Field(alias="LANGFUSE_PUBLIC_KEY")
    langfuse_secret_key: str = Field(alias="LANGFUSE_SECRET_KEY")
    langfuse_host: str = Field(default="http://localhost:3000", alias="LANGFUSE_HOST")
    qdrant_url: str = Field(default="http://localhost:6333", alias="QDRANT_URL")

settings = Settings()
"""

with open("src/core/config.py", "w", encoding="utf-8") as f:
    f.write(config_code.strip())

# 2. Tworzenie .env.example
with open(".env.example", "w") as f:
    f.write("GEMINI_API_KEY=\\nLANGFUSE_PUBLIC_KEY=\\nLANGFUSE_SECRET_KEY=\\nLANGFUSE_HOST=http://localhost:3000\\nQDRANT_URL=http://localhost:6333")

print("LOG: Plik config.py oraz .env.example są gotowe. Synchronizuję środowisko...")

# 3. Synchronizacja uv
!uv sync

## STATUS: SETUP ZAKOŃCZONY

System posiada teraz kompletną infrastrukturę operacyjną.

**Następne kroki:**
1. Ręcznie stwórz plik `.env` w katalogu głównym (na wzór `.env.example`).
2. Uruchom kontenery: `docker compose up -d`.
3. Przejdź do notatnika `01_MULTIMODALNY_ANALITYK.ipynb`.